## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=83e6076214c6aa8e718c5c88871d72a81834f1266ce69ae471cbea70d23c1ed2
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ba638c0f55b90bfa49a5a7c672433f86763252546cfbe0101b8b776602469392
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4863 - accuracy: 0.8309 - val_loss: 0.3677 - val_accuracy: 0.8642
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3585 - accuracy: 0.8677 - val_loss: 0.3582 - val_accuracy: 0.8725
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3246 - accuracy: 0.8807 - val_loss: 0.3828 - val_accuracy: 0.8578


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4178 - accuracy: 0.8502 - val_loss: 0.2985 - val_accuracy: 0.8888
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2748 - accuracy: 0.8991 - val_loss: 0.2675 - val_accuracy: 0.9022
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2243 - accuracy: 0.9158 - val_loss: 0.2797 - val_accuracy: 0.9015


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4766 - accuracy: 0.8297 - val_loss: 0.4327 - val_accuracy: 0.8385
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3488 - accuracy: 0.8695 - val_loss: 0.3890 - val_accuracy: 0.8620
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3070 - accuracy: 0.8849 - val_loss: 0.3769 - val_accuracy: 0.8713


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3923 - accuracy: 0.8584 - val_loss: 0.2927 - val_accuracy: 0.8912
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2475 - accuracy: 0.9086 - val_loss: 0.2534 - val_accuracy: 0.9108
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1867 - accuracy: 0.9296 - val_loss: 0.2548 - val_accuracy: 0.9105


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3974 - accuracy: 0.8566 - val_loss: 0.3184 - val_accuracy: 0.8857
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2633 - accuracy: 0.9023 - val_loss: 0.2877 - val_accuracy: 0.8955
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2088 - accuracy: 0.9229 - val_loss: 0.2602 - val_accuracy: 0.9060


INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        134448    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,110,682
Trainable params: 3,110,682
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1856 - accuracy: 0.9309 - val_loss: 0.2559 - val_accuracy: 0.9123
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1370 - accuracy: 0.9485 - val_loss: 0.2899 - val_accuracy: 0.9107
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0973 - accuracy: 0.9635 - val_loss: 0.2754 - val_accuracy: 0.9175
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0681 - accuracy: 0.9755 - val_loss: 0.3759 - val_accuracy: 0.9125
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0507 - accuracy: 0.9815 - val_loss: 0.3726 - val_accuracy: 0.9148
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0381 - accuracy: 0.9863 - val_loss: 0.4800 - val_accuracy: 0.9123
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0294 - accuracy: 0.9891 - val_loss: 0.5450 - val_a